# COPD Hospital Admission Prediction Project

This project's goal is to predict the likelihood of hospital admission for patients with Chronic Obstructive Pulmonary Disease (COPD).
By implementing machine learning, we collect patient severity levels on symptoms and medical history that can assist healthcare professionals
in decision-making.

## Goals:
1. Preprocess the dataset to make it ready for machine learning.
2. Train a logistic regression model to predict hospital admissions.
3. Evaluate the model using metrics such as precision, recall, F1-score, and ROC-AUC.
4. Save the trained model for deployment.



## PART 1: Import Libraries
Load all needed libraries for app to function

In [ ]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import joblib


## PART 2: Load the Dataset
This part loads the preprocessed dataset and displays the first five rows to view the current data structure.


In [ ]:
# Dynamically locate the project root and construct the file path
notebook_dir = os.path.abspath(os.getcwd())
base_dir = os.path.abspath(os.path.join(notebook_dir, '..'))  # Go up one directory
data_file = os.path.join(base_dir, "data", "copd_data_preprocessed.csv")

# Load the data
try:
    df = pd.read_csv(data_file)
    print("Data loaded successfully!")
    display(df.head())
except FileNotFoundError as e:
    print(f"Error: {e}")
    print(f"Ensure the file exists at: {data_file}")


## PART 3: Preprocess the Data
This part preprocesses the dataset, including splitting into training, testing sets, and scaling the fields to ingest into the machine learning model.


In [ ]:
# Splitting features and target variable
X = df.drop(columns=["Hospital_Admission"])  # Replace with actual target column name
y = df["Hospital_Admission"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preprocessing completed.")


## PART 4: Train the Model
This part includes training the logistic regression model on the provided dataset and prepares it for analysis.


In [ ]:
# Initializing and training the logistic regression model
model = LogisticRegression(
    class_weight="balanced",  # Handling imbalanced data
    max_iter=5000,
    solver="liblinear"
)
model.fit(X_train_scaled, y_train)

print("Model training completed.")


## PART 5: Evaluate the Model
This part evaluates the model using accuracy, precision, recall, and F1-score metrics. It also generates a confusion matrix and its values.


In [ ]:
# Making predictions
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probability scores

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:")
print(class_report)

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC Score: {roc_auc}")


## PART 6: Save the Model
Ready the model by saving for future deployment.

In [ ]:
# Saving the trained model for deployment
model_file = os.path.join(base_dir, "model", "logistic_model.pkl")

try:
    os.makedirs(os.path.join(base_dir, "model"), exist_ok=True)  # Ensure the directory exists
    joblib.dump(model, model_file)
    print(f"Model saved at: {model_file}")
except Exception as e:
    print(f"Error saving model: {e}")


## PART 7: Load Model
Test that the model is functional after saving and does not require retraining.


In [ ]:
# Loading the saved model
try:
    loaded_model = joblib.load(model_file)
    test_proba = loaded_model.predict_proba(X_test_scaled)[:, 1]
    print("Model loaded successfully.")
    print(f"Sample Prediction Probability: {test_proba[:5]}")
except Exception as e:
    print(f"Error loading model: {e}")


## Part 8: Visit the UI on Heroku
https://copd-hospital-assessment-2902c73cfbc3.herokuapp.com/